In [4]:
import spacy
from thinc.api import set_gpu_allocator, require_gpu

import pandas as pd
import re
from tqdm import tqdm
import string
import tarfile
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# load Note datasets
df_notes = pd.read_csv('/home/sdodl001/Downloads/Repos_0/NLP/data/MIMIC/NOTEEVENTS.csv.gz')
df_notes.CATEGORY.unique()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [3]:
categories = df_notes.CATEGORY.unique()
num_samples = {}
for cat in categories:
    print(cat)
    cat_df = df_notes[df_notes.CATEGORY==cat]
    num_samples[cat] = cat_df.shape[0]
    
df_num = pd.DataFrame(num_samples)

Discharge summary
Echo
ECG
Nursing
Physician 
Rehab Services
Case Management 
Respiratory 
Nutrition
General
Social Work
Pharmacy
Consult
Radiology
Nursing/other


{'Discharge summary': 59652,
 'Echo': 45794,
 'ECG': 209051,
 'Nursing': 223556,
 'Physician ': 141624,
 'Rehab Services': 5431,
 'Case Management ': 967,
 'Respiratory ': 31739,
 'Nutrition': 9418,
 'General': 8301,
 'Social Work': 2670,
 'Pharmacy': 103,
 'Consult': 98,
 'Radiology': 522279,
 'Nursing/other': 822497}

In [10]:
num_samples

{'Discharge summary': 59652,
 'Echo': 45794,
 'ECG': 209051,
 'Nursing': 223556,
 'Physician ': 141624,
 'Rehab Services': 5431,
 'Case Management ': 967,
 'Respiratory ': 31739,
 'Nutrition': 9418,
 'General': 8301,
 'Social Work': 2670,
 'Pharmacy': 103,
 'Consult': 98,
 'Radiology': 522279,
 'Nursing/other': 822497}

In [21]:
cats = list(num_samples.keys())
vals = list(num_samples.values())
df_num = pd.DataFrame({'category':cats, 'num_notes':vals})
df_num

,category,num_notes
0,Discharge summary,59652
1,Echo,45794
2,ECG,209051
3,Nursing,223556
4,Physician,141624
5,Rehab Services,5431
6,Case Management,967
7,Respiratory,31739
8,Nutrition,9418
9,General,8301


In [24]:
hist = df_num.hist(bins=3)
hist.show()

AttributeError: 'numpy.ndarray' object has no attribute 'show'

In [25]:
df_num.to_csv('category_num_notes.csv', index=False)

In [ ]:
# load Note datasets
df_notes = pd.read_csv('/home/sdodl001/Downloads/Repos_0/NLP/data/MIMIC/NOTEEVENTS.csv.gz', nrows=10000)
print(df_notes.CATEGORY.unique())
df_notes

['Discharge summary']


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
9995,7953,61943,105572,2104-08-23,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2104-8-17**] ...
9996,7954,11280,143558,2131-04-11,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2131-3-26**] Dischar...
9997,7955,15055,173096,2158-05-19,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2158-4-12**] Dischar...
9998,7956,66014,148736,2170-06-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2170-6-12**] ...


In [ ]:
len(df_notes.ROW_ID.unique())

10000

In [ ]:
df_notes.TEXT.iloc[250]

"Admission Date:  [**2149-9-2**]       Discharge Date:\n\nDate of Birth:   [**2081-3-23**]       Sex:  M\n\nService:\n\nCHIEF COMPLAINT:  Shortness of breath, cough, increased\ndyspnea on exertion.\n\nHISTORY OF PRESENT ILLNESS:  This is a 68 year old male with\na history of congestive heart failure and an ejection\nfraction of 30 to 40%, status post three vessel coronary\nartery bypass graft, likely chronic obstructive pulmonary\ndisease, and a [**2149-5-25**] admission for pansensitive\nStreptococcus pneumococcal pneumonia who presented yesterday\nwith shortness of breath for two days, a cough productive of\nwhite sputum, chills, and increased dyspnea on exertion.  The\npatient was in his usual state of health, able to walk ?????? mile\nbefore becoming winded until three to four days ago when he\ndeveloped rhinorrhea and congestion.  Then one day later he\ndeveloped shortness of breath, cough, chills and increased\ndyspnea on exertion.  He also commenced sharp, substernal\nnonradiati

In [59]:
#  Preprocessing
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    # y=re.sub('?','',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    
    # remove, digits, spaces
    y = y.translate(str.maketrans("", "", string.digits))
    y = " ".join(y.split())
    return y

def preprocessing(df_notes): 
    df_notes['TEXT']=df_notes['TEXT'].fillna(' ')
    df_notes['TEXT']=df_notes['TEXT'].str.replace('\n',' ')
    df_notes['TEXT']=df_notes['TEXT'].str.replace('\r',' ')
    df_notes['TEXT']=df_notes['TEXT'].apply(str.strip)
    df_notes['TEXT']=df_notes['TEXT'].str.lower()

    df_notes['TEXT']=df_notes['TEXT'].apply(lambda x: preprocess1(x))
    
    return df_notes

df_processed = preprocessing(df_notes)
df_processed

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,service: addendum: radiologic studies: radiolo...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: sex: f service: micu and then t...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,service: cardiothoracic allergies: amlodipine ...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,service: medicine allergies: amlodipine attend...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: sex: m service: medicine allerg...
...,...,...,...,...,...,...,...,...,...,...,...
9995,7953,61943,105572,2104-08-23,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: sex: f service: surgery allergi...
9996,7954,11280,143558,2131-04-11,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: sex: m service: medical icu thi...
9997,7955,15055,173096,2158-05-19,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: sex: f service: cardiothoracic ...
9998,7956,66014,148736,2170-06-15,NaN,NaN,Discharge summary,Report,NaN,NaN,date of birth: sex: m service: medicine allerg...


In [58]:
df_processed.TEXT.iloc[250]

"date of birth: sex: m service: chief complaint: shortness of breath, cough, increased dyspnea on exertion. history of present illness: this is a year old male with a history of congestive heart failure and an ejection fraction of to %, status post three vessel coronary artery bypass graft, likely chronic obstructive pulmonary disease, and a admission for pansensitive streptococcus pneumococcal pneumonia who presented yesterday with shortness of breath for two days, a cough productive of white sputum, chills, and increased dyspnea on exertion. the patient was in his usual state of health, able to walk ?????? mile before becoming winded until three to four days ago when he developed rhinorrhea and congestion. then one day later he developed shortness of breath, cough, chills and increased dyspnea on exertion. he also commenced sharp, substernal nonradiating chest pain which he rated out of in intensity. this pain was relieved with rest and worsened with cough. he did have one episode of

In [54]:
# Notes to Sentences
from spacy.lang.en import English
nlp = English()  # just the language with no model
nlp.add_pipe('sentencizer')
# nlp.add_pipe(nlp.create_pipe('sentencizer'))

# nlp praser may not work when there is only one token. In these cases, we just remove them as note that has length 1 usually is some random stuff

def toSentence(x):
    doc = nlp(x)
    text=[]
    try:
        for sent in doc.sents:
            st=str(sent).strip() 
            if len(st)<20:
                #a lot of abbreviation is segmented as one line. But these are all describing the previous things
                #so I attached it to the sentence before
                if len(text)!=0:
                    text[-1]=' '.join((text[-1],st))
                else:
                    text=[st]
            else:
                text.append((st))
    except:
        print(doc)
    return text

sents=df_processed['TEXT'].apply(lambda x: toSentence(x))

In [42]:
# Create Pretraining File
file=open('/home/sdodl001/Downloads/Repos_0/NLP/data/MIMIC/clinical_sentences_pretrain.txt','w')
pretrain_sents = sents.values
for i in tqdm(range(len(pretrain_sents))):
    if len(pretrain_sents[i]) > 0:
        # remove the one token note
        note = pretrain_sents[i]
        for sent in note:
            file.write(sent+'\n')
        file.write('\n')
print('Preprocessed data is saved')

100%|██████████| 10000/10000 [00:00<00:00, 22263.61it/s]

Preprocessed data is saved


In [55]:
df_sents = pd.DataFrame(sents)
df_sents.columns = ['sentences']
df_sents

,sentences
0,[service: addendum: radiologic studies: radiol...
1,[date of birth: sex: f service: micu and then ...
2,[service: cardiothoracic allergies: amlodipine...
3,[service: medicine allergies: amlodipine atten...
4,[date of birth: sex: m service: medicine aller...
...,...
9995,[date of birth: sex: f service: surgery allerg...
9996,[date of birth: sex: m service: medical icu th...
9997,[date of birth: sex: f service: cardiothoracic...
9998,[date of birth: sex: m service: medicine aller...


In [56]:
df_sents.sentences.iloc[250]

['date of birth: sex: m service: chief complaint: shortness of breath, cough, increased dyspnea on exertion.',
 'history of present illness: this is a year old male with a history of congestive heart failure and an ejection fraction of to %, status post three vessel coronary artery bypass graft, likely chronic obstructive pulmonary disease, and a admission for pansensitive streptococcus pneumococcal pneumonia who presented yesterday with shortness of breath for two days, a cough productive of white sputum, chills, and increased dyspnea on exertion.',
 'the patient was in his usual state of health, able to walk ??????',
 'mile before becoming winded until three to four days ago when he developed rhinorrhea and congestion.',
 'then one day later he developed shortness of breath, cough, chills and increased dyspnea on exertion.',
 'he also commenced sharp, substernal nonradiating chest pain which he rated out of in intensity.',
 'this pain was relieved with rest and worsened with cough.',